# [HW2] Topic Modeling
1. Crawling News
2. Preprocessing
3. Build Term-Document Matrix
4. Topic modeling
5. Visualization

```
🔥 이번 시간에는 Topic Modeling를 직접 크롤링한 뉴스 데이터에 대해서 수행해보는 시간을 갖겠습니다. 

먼저 네이버에서 뉴스 기사를 간단하게 크롤링합니다.
기본적인 전처리 이후 Term-document Matrix를 만들고 이를 non-negative factorization을 이용해 행렬 분해를 하여 Topic modeling을 수행합니다.

t-distributed stochastic neighbor embedding(T-SNE) 기법을 이용해 Topic별 시각화를 진행합니다.
```

In [1]:
!pip install newspaper3k
!pip install konlpy

     |████████████████████████████████| 211 kB 12.4 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 25.7 MB/s  eta 0:00:01
     |████████████████████████████████| 7.4 MB 82.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 46.5 MB/s eta 0:00:01
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=1da380693d3f4f796cab5859827408f31b87946d9fe61b4e16698534dcbf7401
  Stored in directory: /home/koowater/.cache/pip/wheels/99/74/83/8fac1c8d9c648cfabebbbffe97a889f6624817f3aa0bbe6c09
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3355 sha256=a7e27263a89b1f45403f6ec03ea8507e6572c6846515d242776d6ebf78effac8
  Stored in directory: /home/koowater/.cache/pip/wheels/b6/09/68/a9f15498ac02c23dde29f18745bc6a6f574ba4ab41861a3575
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398406 sha256=f8b16c73c3cc9ef807024c196b12e068b9dbe036db12165b5d45b7892d3c883b
  Stored in dire

In [2]:
# 크롤링에 필요한 패키지 설치
from bs4 import BeautifulSoup
from newspaper import Article
from time import sleep
from time import time
from dateutil.relativedelta import relativedelta
from datetime import datetime
from multiprocessing import Pool
import json
import requests
import re
import sys

```
💡 Crawling(크롤링)이란?

크롤링은 웹 페이지에서 필요한 데이터를 추출해내는 작업을 말합니다.
이번 시간에는 정적 페이지인 네이버의 뉴스 신문 기사 웹페이지를 크롤링합니다.

HTML은 설명되어 있는 자료가 많기 때문에 생략하도록 하겠습니다.
HTML 구조 파악 및 태그에 대한 설명은 아래 참고자료를 살펴봐주세요 !
```

참고: [위키피디아: 정적페이지](https://ko.wikipedia.org/wiki/%EC%A0%95%EC%A0%81_%EC%9B%B9_%ED%8E%98%EC%9D%B4%EC%A7%80)

참고: [생활코딩: HTML](https://opentutorials.org/course/2039)

In [3]:
def crawl_news(query: str=None, crawl_num: int=1000, workers: int=4):
    '''뉴스 기사 텍스트가 담긴 list를 반환합니다.

    Keyword arguments:
    query -- 검색어 (default None)
    crawl_num -- 수집할 뉴스 기사의 개수 (defualt 1000)
    workers -- multi-processing시 사용할 thread의 개수 (default 4)
    '''

    url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'
    articleList = []
    crawled_url = set()
    keyboard_interrupt = False
    t = time()
    idx = 0
    page = 1

    
    # 서버에 url 요청의 결과를 선언
    res = requests.get(url.format(query))
    sleep(0.5)
    # res를 parsing할 parser를 선언
    bs = BeautifulSoup(res.text, 'html.parser')
    
    with Pool(workers) as p:
        while idx < crawl_num:            
            table = bs.find('ul', {'class': 'list_news'})
            li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
            area_list = [li.find('div', {'class':'news_area'}) for li in li_list]
            a_list = [area.find('a', {'class':'news_tit'}) for area in area_list]
            
            for n in a_list[:min(len(a_list), crawl_num-idx)]:
                articleList.append(n.get('title'))
                idx += 1
            page += 1

            pages = bs.find('div', {'class': 'sc_page_inner'})
            next_page_url = [p for p in pages.find_all('a') if p.text == str(page)][0].get('href')

            req = requests.get('https://search.naver.com/search.naver' + next_page_url)
            bs = BeautifulSoup(req.text, 'html.parser')
    return articleList

```
🔥 이제 '구글'이라는 이름으로 뉴스 기사 1000개의 제목을 크롤링하겠습니다.
```

In [4]:
query = '구글'

articleList = crawl_news(query)

In [5]:
articleList[:10]

['메가존소프트, 구글 클라우드 ‘올해의 아태 세일즈 파트너상’ 수상',
 '구글 러시아 법인 파산신청…"계좌 압류로 기업활동 불가"',
 '메타·구글·트위터 등 EU ‘허위정보 대응 강화’ 서명',
 '조선대 22일부터 ON-AIR 직무박람회…네이버·구글 등 현직자 참여',
 '‘임금 성차별’ 소송당한 구글, 여직원들에게 1,515억 원 지급 합의',
 "인앱결제 두고 깊어지는 카카오-구글 '눈치싸움'…향후 전망은?",
 "메타·구글·트위터 등 '허위정보 대응 강화' 약속",
 '여성 임금 적게 준 구글, 1516억원 합의금 지급한다',
 "[친절한B씨]달라진 구글, '인앱결제 이슈'로 뚜렷해진 전략 변화",
 '‘임금 성차별’ 소송 걸린 구글, 여성 직원에 1515억원 지불키로']

```
🔥 태거(tagger)를 이용해 한글 명사와 알파벳만을 추출해서 term-document matrix (tdm)을 만들겠습니다.

태거(tagger)는 tokenization에서 조금 더 자세히 다루도록 하겠습니다.
```

참고: [konlpy: morph analyzer](https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/)

## Preprocessing

In [18]:
from konlpy.tag import Okt
from collections import Counter
import json

# Okt 형태소 분석기 선언
t = Okt()

words_list_ = []
vocab = Counter()
tag_set = set(['Noun', 'Alpha'])
stopwords = set(['글자'])

for i, article in enumerate(articleList):
    if i % 100 == 0:
        print(i)
    
    # tagger를 이용한 품사 태깅
    words = t.pos(article, norm=True, stem=True)

    ############################ ANSWER HERE ################################
    # TODO: 다음의 조건을 만족하는 단어의 리스트를 완성하세요.
    # 조건 1: 명사와 알파벳 tag를 가진 단어
    # 조건 2: 철자 길이가 2이상인 단어 
    # 조건 3: stopwords에 포함되지 않는 단어
    #########################################################################        
    
    words = [word[0] for word in words 
        if word[1] in tag_set 
        and len(word[0]) >= 2 
        and word[0] not in stopwords]
        
    vocab.update(words)
    words_list_.append((words, article))
    
vocab = sorted([w for w, freq in vocab.most_common(10000)])
word2id = {w: i for i, w in enumerate(vocab)}
words_list = []
for words, article in words_list_:
    words = [w for w in words if w in word2id]
    if len(words) > 10:
        words_list.append((words, article))
        
del words_list_

0
100
200
300
400
500
600
700
800
900


## Build document-term matrix

```
🔥 이제 document-term matrix를 만들어보겠습니다.
document-term matrix는 (문서 개수 x 단어 개수)의 Matrix입니다.
```

참고: [Document-Term Matrix](https://wikidocs.net/24559)

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np

dtm = np.zeros((len(words_list), len(vocab)), dtype=np.float32)
for i, (words, article) in enumerate(words_list):
    for word in words:
        dtm[i, word2id[word]] += 1
        
dtm = TfidfTransformer().fit_transform(dtm)

```
🔥 document-term matrix를 non-negative factorization(NMF)을 이용해 행렬 분해를 해보겠습니다.

💡 Non-negative Factorization이란?

NMF는 주어진 행렬 non-negative matrix X를 non-negative matrix W와 H로 행렬 분해하는 알고리즘입니다.
이어지는 코드를 통해 W와 H의 의미에 대해 파악해봅시다.
```
참고: [Non-negative Matrix Factorization](https://angeloyeo.github.io/2020/10/15/NMF.html)

## Topic modeling

In [20]:
# Non-negative Matrix Factorization
from sklearn.decomposition import NMF

K = 5
nmf = NMF(n_components=K, alpha=0.1)

```
🔥 sklearn의 NMF를 이용해 W와 H matrix를 구해봅시다.
W는 document length x K, H는 K x term length의 차원을 갖고 있습니다.
W의 하나의 row는 각각의 feature에 얼만큼의 가중치를 줄 지에 대한 weight입니다.
H의 하나의 row는 하나의 feature를 나타냅니다.

우선 하나의 Topic (H의 n번째 row)에 접근해서 해당 topic에 대해 값이 가장 높은 20개의 단어를 출력해보겠습니다.
```

In [21]:
W = nmf.fit_transform(dtm)
H = nmf.components_

/home/koowater/.local/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1477: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(


```
🔥 우선 하나의 Topic (H의 n번째 row)에 접근해서 해당 topic에 대해 값이 가장 높은 20개의 단어를 출력해보겠습니다.
```

In [22]:
for k in range(K):
    print(f"{k}th topic")
    for index in H[k].argsort()[::-1][:20]:
        print(vocab[index], end=' ')
    print()

0th topic
결제 인앱 강제 구글 방통위 가격 링크 아웃 유지 불응 방침 카카오 현실 기존 안드로이드 라면 대신 이용자 라운드 콘텐츠 
1th topic
완료 승인 버스 타운 싸이 메타 서비스 한컴 출시 연동 싸이월드 정식 눈앞 구글 코앞 한글과컴퓨터 애플 마켓 증권 스토어 
2th topic
삼성 지원 전자 요구 목소리 통과 반도체 인텔 미국 MS 구글 갤럭시 어시스턴트 워치 협력 강화 시작 관계 웨어 러블 
3th topic
아마존 AI LG 유플러스 데이터 디지털 혁신 기업 전환 구글 클라우드 수익 현장 처럼 가속 기반 박차 TV 웨이 서비스 
4th topic
시민 회의 소비자주권 고발 형사 행위 오늘 강제 인앱 결제 구글 핑계 줄줄이 중단 즉각 요금 인상 기업 콘텐츠 단체 


```
🔥 이번에는 W에서 하나의 Topic (W의 n번째 column)에 접근해서 해당 topic에 대해 값이 가장 높은 3개의 뉴스 기사 제목을 출력해보겠습니다.
```

In [23]:
for k in range(K):
    print(f"==={k}th topic===")
    for index in W[:, k].argsort()[::-1][:3]:
        print(words_list[index][1])
    print('\n')

===0th topic===
카카오, 구글 인앱결제 강제방침 불응…웹 결제 아웃링크 유지
카카오, 구글 인앱결제 강제방침 불응…웹 결제 아웃링크 유지
구글 인앱결제 강제 현실화…"안드로이드 이용자라면 앱 대신 웹에서 기존 가격으로 결제하세요"


===1th topic===
한컴, 메타버스 서비스 '싸이타운' 정식 출시 눈앞에...구글 앱 승인 완료
한컴, 메타버스 서비스 '싸이타운' 정식 출시 눈앞에...구글 앱 승인 완료
한컴, 메타버스 서비스 '싸이타운' 구글 앱 승인 완료...."싸이월드와 연동만 남았다"


===2th topic===
삼성전자 미국 반도체 지원법 통과 요구, 인텔 구글 MS와 ‘한목소리’
삼성전자 미국 반도체 지원법 통과 요구, 인텔 구글 MS와 ‘한목소리’
삼성전자 미국 반도체 지원법 통과 요구, 인텔 구글 MS와 ‘한목소리’


===3th topic===
LG유플러스 "구글·아마존처럼 데이터· AI로 수익 창출...디지털 혁신기업 전환”
[A 현장] "구글·아마존 처럼" LG유플러스, 데이터·AI로 디지털 혁신기업 전환 가속화
[뉴스웨이TV]"구글·아마존처럼" LG유플러스, 데이터·AI로 디지털 혁신 박차


===4th topic===
소비자주권시민회의, 오늘 ‘구글 인앱결제 강제행위’ 형사고발
소비자주권시민회의, 오늘 ‘구글 인앱결제 강제행위’ 형사고발
소비자주권시민회의, 오늘 ‘구글 인앱결제 강제행위’ 형사고발




```
❓ 2번째 토픽에 대해 가장 높은 가중치를 갖는 제목 5개를 출력해볼까요?
```

In [24]:
# TODO
index_list = W[:, 2].argsort()[::-1][:5]
for index in index_list:
    print(words_list[index][1])

삼성전자 미국 반도체 지원법 통과 요구, 인텔 구글 MS와 ‘한목소리’
삼성전자 미국 반도체 지원법 통과 요구, 인텔 구글 MS와 ‘한목소리’
삼성전자 미국 반도체 지원법 통과 요구, 인텔 구글 MS와 ‘한목소리’
삼성전자, '갤럭시 워치 4'에 구글 어시스턴트 지원 시작... 구글 협력 관계 강화!
갤럭시워치4, 구글 어시스턴트 지원... 삼성-구글 웨어러블 협력 강화


```
🔥 이번에는 t-SNE를 이용해 Topic별 시각화를 진행해보겠습니다.

💡 t-SNE는 무엇인가요?

t-Stochastic Neighbor Embedding(t-SNE)은 고차원의 벡터를 
저차원(2~3차원) 벡터로 데이터간 구조적 특징을 유지하며 축소를 하는 방법 중 하나입니다.

주로 고차원 데이터의 시각화를 위해 사용됩니다.
```

참고: [lovit: t-SNE](https://lovit.github.io/nlp/representation/2018/09/28/tsne/#:~:text=t%2DSNE%20%EB%8A%94%20%EA%B3%A0%EC%B0%A8%EC%9B%90%EC%9D%98,%EC%9D%98%20%EC%A7%80%EB%8F%84%EB%A1%9C%20%ED%91%9C%ED%98%84%ED%95%A9%EB%8B%88%EB%8B%A4.)

참고: [ratsgo: t-SNE](https://ratsgo.github.io/machine%20learning/2017/04/28/tSNE/)

## Visualization

In [27]:
from sklearn.manifold import TSNE

# n_components = 차원 수
tsne = TSNE(n_components=2, init='pca', verbose=1)

# W matrix에 대해 t-sne를 수행합니다.
W2d = tsne.fit_transform(W)

# 각 뉴스 기사 제목마다 가중치가 가장 높은 topic을 저장합니다.
topicIndex = [v.argmax() for v in W]

/home/koowater/.local/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/home/koowater/.local/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:991: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


[t-SNE] Computing 79 nearest neighbors...
[t-SNE] Indexed 80 samples in 0.000s...
[t-SNE] Computed neighbors for 80 samples in 0.038s...
[t-SNE] Computed conditional probabilities for sample 80 / 80
[t-SNE] Mean sigma: 0.149095
[t-SNE] KL divergence after 250 iterations with early exaggeration: 52.321682
[t-SNE] KL divergence after 1000 iterations: -0.269322


In [29]:
!pip install bokeh

     |████████████████████████████████| 18.5 MB 13.6 MB/s eta 0:00:01


In [30]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
output_notebook()

# 사용할 툴들
tools_to_show = 'hover,box_zoom,pan,save,reset,wheel_zoom'
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article for words, article in words_list],
    'topic': [str(i) for i in topicIndex],  # 토픽 번호
    'color': [Category20[K][i] for i in topicIndex]
})
p.circle(
    'x', 'y',
    source=source,
    legend='topic',
    color='color'
)

# interaction
p.legend.location = "top_left"
hover = p.select({'type': HoverTool})
hover.tooltips = [("Topic", "@topic"), ('id', '@id'), ("Article", "@document")]
hover.mode = 'mouse'

show(p)

Loading BokehJS ...